In [1]:
import numpy as np
import random
import math
import copy
import pandas as pd

# Define global variables

In [244]:
have_model = False

In [322]:
ttt_combos = [[0,1,2],[3,4,5],[6,7,8],
              [0,3,6],[1,4,7],[2,5,8],
              [0,4,8],[2,4,6]]

init_board = [[-1 for _ in range(9)] for _ in range(9)]

state_conv = {
    -1 : ' ',
    0 : 'O',
    1 : 'X'
}

# Setting up the game

In [466]:
# define a node for game tree used in the MCTS
class Node:
    def __init__(self, state, 
                 small_board = -1, parent = None):
        self.state = state
        self.small_board = small_board # the small board current player is sent to
        self.parent = parent
        self.children = []
        self.value = 0
        self.visits = 0
        self.current_player = 1
        if count(self.state, 1) > count(self.state, 0):
            self.current_player = 0
        self.legal_moves = self.get_possible_moves()
    
    # play a move
    def play(self, move):
        if move in self.legal_moves:
            new_state = copy.deepcopy(self.state)
            new_state[move[0]][move[1]] = self.current_player
            s_board = 3*(move[0] % 3) + (move[1] % 3)

            new_node = Node(new_state, s_board, self)
            self.children.append(new_node)
            return new_node
        else:
            raise ValueError('Not a legal move')
    
    # returns what the next state would be if the move is played
    def next_state(self, move):
        new_state = copy.deepcopy(self.state)
        new_state[move[0]][move[1]] = self.current_player
        return new_state
    
    # returns whether the state represents a game that has ended
    def is_terminal(self):
        sb_states = self.get_small_board_states()
        for l in ttt_combos:
            if sb_states[l[0]] == sb_states[l[1]] == sb_states[l[2]] and sb_states[l[0]] != -1 and sb_states[l[0]] != 2:
                return True
        if -1 not in sb_states:
            return True
        return False
    
    # determine the status of each of the 9 small boards
    # -1 : open board, 0 : O wins, 1 : X wins, 2 : Tie 
    def get_small_board_states(self):
        states = [-1 for _ in range(9)]
        for k in range(9):
            s_board = []
            row_lower = 3*(k // 3)
            col_lower = 3*(k % 3)
            for i in range(row_lower, row_lower + 3):
                for j in range(col_lower, col_lower + 3):
                    s_board.append(self.state[i][j])
            for l in ttt_combos:
                if s_board[l[0]] == s_board[l[1]] == s_board[l[2]] and s_board[l[0]] != -1:
                    states[k] = s_board[l[0]]
                    break
            if -1 not in s_board and states[k] == -1:
                states[k] = 2
        return states
    
    # get legal moves from current node
    def get_possible_moves(self):
        if self.small_board == -1: # only applies to initial state
            return [[i, j] for i in range(9) for j in range(9)]
        
        if self.is_terminal():
            return []

        small_board_states = self.get_small_board_states()

        if small_board_states[self.small_board] != -1:
            legal_moves = []
            for k in range(9):
                if small_board_states[k] == -1:
                    row_lower = 3*(k // 3)
                    col_lower = 3*(k % 3) 
                    legal_moves.extend([[i, j] for i in range(row_lower,row_lower+3) for j in range(col_lower,col_lower+3) if self.state[i][j] == -1])
        else:
            row_lower = 3*(self.small_board // 3)
            col_lower = 3*(self.small_board % 3) 
            legal_moves = [[i, j] for i in range(row_lower,row_lower+3) for j in range(col_lower,col_lower+3) if self.state[i][j] == -1]

        return legal_moves
            
    # selects node with unexplored moves
    def select(self, exp_weight = 2):
        node = self
        
        # continue until leaf node (one with unexplored moves)
        while (not node.get_unexplored_moves()) and node.legal_moves:
            node = node.best_ucb(exp_weight)
        return node
    
    # get unexplored moves from current node
    def get_unexplored_moves(self):
        return [move for move in self.legal_moves if self.next_state(move) not in [child.state for child in self.children]]

    # picks out the child node with the best UCB
    def best_ucb(self, exp_weight):
        if have_model:
            predictions = model([np.array([self.state]), np.array([self.small_board])])
            policy = predictions[0][0]
            return max(self.children, key = lambda child : child.ucb(exp_weight * policy[get_flat_move(self, child)]))
        else:
            return max(self.children, key = lambda child : child.ucb(exp_weight))

    # calculates the UCB of a node
    def ucb(self, exp_weight):
        return self.value / self.visits + exp_weight * math.sqrt(math.log(self.parent.visits) / self.visits)
    
    # get nodes corresponding to unexplored move
    def expand(self):
        unexplored_moves = self.get_unexplored_moves()
        if unexplored_moves: # if there are unexplored moves
            move = random.choice(unexplored_moves)
            return self.play(move)
        else:
            return self  # No unexplored moves, return the current node
    
    # play out a game from current node
    def simulate(self):
        node = self
        if have_model:
            for _ in range(depth):
                if not node.is_terminal():
                    move = random.choice(node.legal_moves) 
                    node = node.play(move)
                else:
                    return node, node.get_result()
            return node, model([np.array([self.state]), np.array([self.small_board])])[1][0][0]
        else:
            while not node.is_terminal():  
                move = random.choice(node.legal_moves) 
                node = node.play(move)
            return node, node.get_result()
    
    # returns winner
    def get_result(self):
        sb_states = self.get_small_board_states()
        for l in ttt_combos:
            if sb_states[l[0]] == sb_states[l[1]] == sb_states[l[2]] and sb_states[l[0]] != -1:
                return sb_states[l[0]]
        return -1
    
    # update nodes based on results of simulation
    def backpropagate(self, result, root):
        node = self
        end_node_player = self.current_player
        while node != root.parent: # continue until root node
            node.visits += 1
            if result == -1:
                node.value += 0.5
            else:
                if have_model:
                    if node.current_player == end_node_player:
                        node.value += result
                    else:
                        node.value += (1 - result)
                elif node.current_player != result:
                    node.value += 1 
            node = node.parent
        
    def __str__(self):
        # gives a text representation of the board using print()
        ret_str = f'Legal moves: {self.legal_moves} \nCurrent player: {state_conv[self.current_player]}\n'
        for i in range(9):
            for j in range(9):
                ret_str += state_conv[self.state[i][j]]
                if j == 2 or j == 5:
                    ret_str += ' '
                if j < 8:
                    ret_str += '|'
                if j == 2 or j == 5:
                    ret_str += ' '
            ret_str += '\n'
            if i == 2 or i == 5:
                ret_str += (' '*5 + ' | ')*2 + ' '*5 + '\n'
                ret_str += ('-'*5 + '-+-')*2 + '-'*5 + '\n'
                ret_str += (' '*5 + ' | ')*2 + ' '*5 + '\n'
            elif i < 8:
                ret_str += ('-+-+-' + ' | ')*2 + '-+-+-' + '\n'
        return ret_str

# Setting up the MCTS

In [467]:
# runs the MCTS
def mcts(root, iterations):
    selected_nodes = []
    for _ in range(iterations):
        current_node = root.select()
        if current_node not in selected_nodes:
            selected_nodes.append(current_node)
        if not current_node.is_terminal():
            current_node = current_node.expand()
        end_node, result = current_node.simulate()
        end_node.backpropagate(result, root)
        
    for selected_node in selected_nodes:
        probs = [0 for _ in range(81)]
        for child in selected_node.children:
            probs[get_flat_move(selected_node, child)] = child.value/child.visits
        total = sum(probs)
        if total != 0:
            probs = [x/total for x in probs]
        
        node_tuple = (tuple(tuple(s) for s in selected_node.state), selected_node.small_board)
        
        if mcts_dict.get(node_tuple) is None:
            mcts_dict[node_tuple] = [selected_node.value, selected_node.visits]
            mcts_dict[node_tuple].append(probs)
        else:
            mcts_dict[node_tuple][0] += selected_node.value
            mcts_dict[node_tuple][1] += selected_node.visits
            mcts_dict[node_tuple][2] = probs
    
    return best_child(root)

# selects the best next action based on the child that is visited the most
def best_child(node):
    return max(node.children, key=lambda child: child.visits)

# counts how many moves the player has made in current state
def count(state, player):
    return sum([substate.count(player) for substate in state])
            
def get_flat_move(parent, child):
    for i in range(9):
        for j in range(9):
            if parent.state[i][j] != child.state[i][j]:
                return 9*i + j

# MCTS iteration 1

In [171]:
# Dictionary for storing nodes selected by MCTS
mcts_dict = {}

In [172]:
# import time
# start_time = time.time()

num_games = 10
for _ in range(num_games):
    root_node = Node(init_board)
    current_node = root_node
    while not current_node.is_terminal():
        current_node = mcts(current_node, iterations=100)
        
# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"Elapsed Time: {elapsed_time} seconds")

Elapsed Time: 130.22808527946472 seconds


In [6]:
mcts_df = pd.DataFrame([{'state' : list(list(s) for s in key[0]), 
                         'small_board' : key[1],
                         'value' : value[0], 
                         'visits' : value[1],
                         'probs' : value[2]
                        } for key, value in mcts_dict.items()])

In [7]:
mcts_df

,state,small_board,value,visits,probs
0,"[[-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1...",-1,5.0,10,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[[-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1...",6,3.5,11,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[[-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1...",0,7.5,14,"[0.25, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
3,"[[-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1...",8,1.0,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[[-1, -1, -1, -1, -1, -1, -1, -1, -1], [-1, -1...",7,10.5,14,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...
230,"[[1, 0, 0, -1, -1, 1, 1, 1, 0], [0, 0, 1, 0, 1...",1,0.0,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
231,"[[1, 0, 0, -1, -1, 1, 1, 1, 0], [0, 0, 1, 0, 1...",1,0.0,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
232,"[[1, -1, 0, -1, -1, 1, 1, 1, 0], [0, 0, 1, 0, ...",0,2.0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
233,"[[1, 0, 0, -1, -1, 1, 1, 1, 0], [0, 0, 1, 0, 1...",5,2.0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [137]:
# saving MCTS information
mcts_df.to_csv('mcts1.csv', index = False)

In [83]:
# loading MCTS information
mcts_df = pd.read_csv('mcts1.csv')

import ast
mcts_df['state'] = mcts_df['state'].apply(ast.literal_eval)
mcts_df['probs'] = mcts_df['probs'].apply(ast.literal_eval)

# Neural network iteration 1

In [130]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2, VGG16
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [124]:
# Load VGG16 pre-trained on ImageNet (include_top=False to exclude the top classification layer)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(36, 36, 3)) 

# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

58889256/58889256 [==============================] - 3s 0us/step


In [131]:
# Input for the 9x9 grid
input_state = layers.Input(shape=(9, 9, 1), name='state')

# Input for the integer representing the next small board
input_small_board = layers.Input(shape=(1,), name='small_board')

scaled_input = layers.Rescaling(scale=1.0/2.0, offset=0.5)(input_state)
resized_input = layers.Lambda(lambda x: tf.image.resize(x, (36, 36)))(scaled_input)  
expanded_input = layers.Concatenate()([resized_input, resized_input, resized_input])  
x = base_model(expanded_input, training=False)

x = layers.Flatten()(x)
x = layers.concatenate([x, input_small_board])
x = layers.Dense(256, activation='relu')(x)

# Output for policy
output_probs = layers.Dense(81, activation='softmax', name='output_probs')(x)

# Output for value
output_value = layers.Dense(1, activation='sigmoid', name='output_value')(x)

# Create and compile the model
model = models.Model(inputs=[input_state, input_small_board], outputs=[output_probs, output_value])
model.compile(optimizer=Adam(0.0001), loss={'output_probs': 'categorical_crossentropy', 'output_value': 'mean_squared_error'})


In [145]:
model.summary()

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 state (InputLayer)          [(None, 9, 9, 1)]            0         []                            
                                                                                                  
 rescaling_1 (Rescaling)     (None, 9, 9, 1)              0         ['state[0][0]']               
                                                                                                  
 lambda_11 (Lambda)          (None, 36, 36, 1)            0         ['rescaling_1[0][0]']         
                                                                                                  
 concatenate_24 (Concatenat  (None, 36, 36, 3)            0         ['lambda_11[0][0]',           
 e)                                                                  'lambda_11[0][0]',    

In [138]:
# create the input/output for the neural network
X = [tf.constant(mcts_df['state'].apply(lambda x : [[[a] for a in s] for s in x]).to_list()), tf.constant(mcts_df['small_board'])]
y = [tf.constant(mcts_df['probs'].apply(lambda x : [[s] for s in x]).to_list()), tf.constant(mcts_df['value']/mcts_df['visits'])]

In [139]:
# train/validation/test split
X1_train, X1_temp, X2_train, X2_temp, y1_train, y1_temp, y2_train, y2_temp = train_test_split(np.array(X[0]), np.array(X[1]), np.array(y[0]), np.array(y[1]), test_size=0.2, random_state=20)
X1_valid, X1_test, X2_valid, X2_test, y1_valid, y1_test, y2_valid, y2_test = train_test_split(X1_temp, X2_temp, y1_temp, y2_temp, test_size=0.5, random_state=20)

In [140]:
model.fit([X1_train, X2_train], [y1_train, y2_train], validation_data=([X1_valid, X2_valid], [y1_valid, y2_valid]), epochs=2)

Epoch 1/2
472/472 [==============================] - 155s 329ms/step - loss: 3.3921 - output_probs_loss: 3.3108 - output_value_loss: 0.0813 - val_loss: 3.4217 - val_output_probs_loss: 3.3381 - val_output_value_loss: 0.0836
Epoch 2/2
472/472 [==============================] - 157s 333ms/step - loss: 3.3942 - output_probs_loss: 3.3134 - output_value_loss: 0.0808 - val_loss: 3.4605 - val_output_probs_loss: 3.3786 - val_output_value_loss: 0.0819


In [419]:
# saving model
model.save('nn1.h5')

C:\Users\bliu9\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [437]:
# loading model
model = models.load_model('nn1.h5', safe_mode=False)

# MCTS and neural network iterations

In [463]:
have_model = True
depth = 10
iterations = 20
num_games = 10

In [401]:
import time

In [ ]:
for i in range(iterations):
    print(f'Starting MCTS #{i+2}')
    start_time = time.time()

    mcts_dict = {}
    for _ in range(num_games):
        root_node = Node(init_board)
        current_node = root_node
        while not current_node.is_terminal():
            current_node = mcts(current_node, iterations=100)
            print(current_node)
            
    mcts_df = pd.DataFrame([{'state' : list(list(s) for s in key[0]), 
                         'small_board' : key[1],
                         'value' : value[0], 
                         'visits' : value[1],
                         'probs' : value[2]
                        } for key, value in mcts_dict.items()])
    
    mcts_df.to_csv(f'mcts{i+2}.csv', index = False)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"MCTS #{i+2} Elapsed Time: {elapsed_time} seconds")
    
    
    
    print(f'Starting neural network #{i+2}')
    start_time = time.time()

    input_state = layers.Input(shape=(9, 9, 1), name='state')
    input_small_board = layers.Input(shape=(1,), name='small_board')
    scaled_input = layers.Rescaling(scale=1.0/2.0, offset=0.5)(input_state)
    resized_input = layers.Lambda(lambda x: tf.image.resize(x, (36, 36)))(scaled_input)  # Resize to meet the minimum input size
    expanded_input = layers.Concatenate()([resized_input, resized_input, resized_input])  # Expand to 3 channels for MobileNetV2
    x = base_model(expanded_input, training=False)
    x = layers.Flatten()(x)
    x = layers.concatenate([x, input_small_board])
    x = layers.Dense(256, activation='relu')(x)
    output_probs = layers.Dense(81, activation='softmax', name='output_probs')(x)
    output_value = layers.Dense(1, activation='sigmoid', name='output_value')(x)
    model = models.Model(inputs=[input_state, input_small_board], outputs=[output_probs, output_value])
    model.compile(optimizer=Adam(0.0001), loss={'output_probs': 'categorical_crossentropy', 'output_value': 'mean_squared_error'})

    X = [tf.constant(mcts_df['state'].apply(lambda x : [[[a] for a in s] for s in x]).to_list()), tf.constant(mcts_df['small_board'])]
    y = [tf.constant(mcts_df['probs'].apply(lambda x : [[s] for s in x]).to_list()), tf.constant(mcts_df['value']/mcts_df['visits'])]
    X1_train, X1_temp, X2_train, X2_temp, y1_train, y1_temp, y2_train, y2_temp = train_test_split(np.array(X[0]), np.array(X[1]), np.array(y[0]), np.array(y[1]), test_size=0.2, random_state=20)
    X1_valid, X1_test, X2_valid, X2_test, y1_valid, y1_test, y2_valid, y2_test = train_test_split(X1_temp, X2_temp, y1_temp, y2_temp, test_size=0.5, random_state=20)
    model.fit([X1_train, X2_train], [y1_train, y2_train], validation_data=([X1_valid, X2_valid], [y1_valid, y2_valid]), epochs=2)
    model.save(f'nn{i+2}.keras')
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Neural network #{i+2} Elapsed Time: {elapsed_time} seconds")

# Example game playing against itself

In [468]:
current_node = Node(init_board)

In [450]:
current_node = current_node.play([8,3])
print(current_node)

if not current_node.is_terminal():
    current_node = mcts(current_node, iterations=100)
    print(current_node)

Legal moves: [] 
Current player: O
O| |X |  | |X | O|O|X
-+-+- | -+-+- | -+-+-
X|O|O | X| |X | O| |O
-+-+- | -+-+- | -+-+-
X|O|X |  | |X | O| |X
      |       |      
------+-------+------
      |       |      
X|O|  | X|O|O | X| |X
-+-+- | -+-+- | -+-+-
 |X|O |  |X|  | X| | 
-+-+- | -+-+- | -+-+-
 | |O |  | |X | X| |O
      |       |      
------+-------+------
      |       |      
X| |  |  | |O | O|O|X
-+-+- | -+-+- | -+-+-
 |O|O |  | |  | O|O| 
-+-+- | -+-+- | -+-+-
 |O|  | X|X|X | X|O| 



In [469]:
while not current_node.is_terminal():
    current_node = mcts(current_node, iterations=100)
    print(current_node)

Legal moves: [[6, 0], [6, 1], [6, 2], [7, 0], [7, 1], [7, 2], [8, 0], [8, 1], [8, 2]] 
Current player: O
 | |  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
 | |  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
X| |  |  | |  |  | | 
      |       |      
------+-------+------
      |       |      
 | |  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
 | |  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
 | |  |  | |  |  | | 
      |       |      
------+-------+------
      |       |      
 | |  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
 | |  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
 | |  |  | |  |  | | 

Legal moves: [[6, 3], [6, 4], [6, 5], [7, 3], [7, 4], [7, 5], [8, 3], [8, 4], [8, 5]] 
Current player: X
 | |  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
 | |  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
X| |  |  | |  |  | | 
      |       |      
------+-------+------
      |       |      
 | |  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
 | |  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
 | |  |  | |  |  | | 
      |       |      
------+-------+----

Legal moves: [[0, 6], [0, 7], [0, 8], [1, 6], [1, 7], [1, 8], [2, 6], [2, 7], [2, 8]] 
Current player: X
 | |  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
X| |  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
X| |X |  | |  |  | | 
      |       |      
------+-------+------
      |       |      
 | |  |  | |O |  | | 
-+-+- | -+-+- | -+-+-
 |X|  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
O| |  |  | |X | O| | 
      |       |      
------+-------+------
      |       |      
O| |  |  | |  | O| | 
-+-+- | -+-+- | -+-+-
 | |X |  | |  | O|O| 
-+-+- | -+-+- | -+-+-
 |O|X | X| |  |  | | 

Legal moves: [[3, 0], [3, 1], [3, 2], [4, 0], [4, 2], [5, 1], [5, 2]] 
Current player: O
 | |  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
X| |  |  | |  | X| | 
-+-+- | -+-+- | -+-+-
X| |X |  | |  |  | | 
      |       |      
------+-------+------
      |       |      
 | |  |  | |O |  | | 
-+-+- | -+-+- | -+-+-
 |X|  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
O| |  |  | |X | O| | 
      |       |      
------+-------+------
      |      

Legal moves: [[3, 1], [3, 2], [4, 0], [4, 2], [5, 1], [5, 2]] 
Current player: O
X| |  |  | |O |  | |X
-+-+- | -+-+- | -+-+-
X| |  | X| |O | X| | 
-+-+- | -+-+- | -+-+-
X| |X |  | |  |  |O| 
      |       |      
------+-------+------
      |       |      
O| |  |  |X|O |  |O| 
-+-+- | -+-+- | -+-+-
 |X|  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
O| |  |  | |X | O|X| 
      |       |      
------+-------+------
      |       |      
O| |  |  | |  | O| | 
-+-+- | -+-+- | -+-+-
 |O|X |  | |X | O|O| 
-+-+- | -+-+- | -+-+-
 |O|X | X| |O | X| | 

Legal moves: [[0, 3], [0, 4], [1, 4], [2, 3], [2, 4], [2, 5]] 
Current player: X
X| |  |  | |O |  | |X
-+-+- | -+-+- | -+-+-
X| |  | X| |O | X| | 
-+-+- | -+-+- | -+-+-
X| |X |  | |  |  |O| 
      |       |      
------+-------+------
      |       |      
O|O|  |  |X|O |  |O| 
-+-+- | -+-+- | -+-+-
 |X|  |  | |  |  | | 
-+-+- | -+-+- | -+-+-
O| |  |  | |X | O|X| 
      |       |      
------+-------+------
      |       |      
O| |  |  | |  | O| | 
-

Legal moves: [[3, 6], [3, 8], [4, 6], [4, 7]] 
Current player: X
X| |  |  |X|O | X|O|X
-+-+- | -+-+- | -+-+-
X| |  | X| |O | X| |O
-+-+- | -+-+- | -+-+-
X| |X | X| |O |  |O| 
      |       |      
------+-------+------
      |       |      
O|O|  |  |X|O |  |O| 
-+-+- | -+-+- | -+-+-
 |X|  |  | |  |  | |O
-+-+- | -+-+- | -+-+-
O| |O |  | |X | O|X|X
      |       |      
------+-------+------
      |       |      
O| |O | X| |  | O|X| 
-+-+- | -+-+- | -+-+-
 |O|X |  | |X | O|O|X
-+-+- | -+-+- | -+-+-
 |O|X | X| |O | X|O| 

Legal moves: [[1, 7], [2, 6], [2, 8]] 
Current player: O
X| |  |  |X|O | X|O|X
-+-+- | -+-+- | -+-+-
X| |  | X| |O | X| |O
-+-+- | -+-+- | -+-+-
X| |X | X| |O |  |O| 
      |       |      
------+-------+------
      |       |      
O|O|  |  |X|O |  |O|X
-+-+- | -+-+- | -+-+-
 |X|  |  | |  |  | |O
-+-+- | -+-+- | -+-+-
O| |O |  | |X | O|X|X
      |       |      
------+-------+------
      |       |      
O| |O | X| |  | O|X| 
-+-+- | -+-+- | -+-+-
 |O|X |  | |X | O|O

Legal moves: [[2, 6]] 
Current player: X
X| |  |  |X|O | X|O|X
-+-+- | -+-+- | -+-+-
X| |  | X| |O | X|X|O
-+-+- | -+-+- | -+-+-
X| |X | X| |O |  |O|O
      |       |      
------+-------+------
      |       |      
O|O|  | O|X|O |  |O|X
-+-+- | -+-+- | -+-+-
X|X|O |  |O|  | X| |O
-+-+- | -+-+- | -+-+-
O|O|O |  |X|X | O|X|X
      |       |      
------+-------+------
      |       |      
O| |O | X|X|O | O|X|O
-+-+- | -+-+- | -+-+-
 |O|X |  |X|X | O|O|X
-+-+- | -+-+- | -+-+-
O|O|X | X| |O | X|O|X

Legal moves: [[4, 3], [4, 5], [5, 3], [3, 6], [4, 7], [7, 3], [8, 4]] 
Current player: O
X| |  |  |X|O | X|O|X
-+-+- | -+-+- | -+-+-
X| |  | X| |O | X|X|O
-+-+- | -+-+- | -+-+-
X| |X | X| |O | X|O|O
      |       |      
------+-------+------
      |       |      
O|O|  | O|X|O |  |O|X
-+-+- | -+-+- | -+-+-
X|X|O |  |O|  | X| |O
-+-+- | -+-+- | -+-+-
O|O|O |  |X|X | O|X|X
      |       |      
------+-------+------
      |       |      
O| |O | X|X|O | O|X|O
-+-+- | -+-+- | -+-+-
 |O|X |  |X